In [1]:
# Importing necessary libraries and modules

# Standard library imports
import asyncio  # For asynchronous operations
import time  # For time-based operations and timestamps
import os  # For environment management and file operations
import warnings  # To filter out warnings
import nest_asyncio  # To avoid runtime errors related to asynchronous operations

# Environment management
from dotenv import load_dotenv

# Third-party library imports
import matplotlib.pyplot as plt  # For plotting and visualization
import numpy as np  # For numerical operations and array manipulations
import pandas as pd  # For data manipulation and analysis
import pandas_ta as ta  # For technical analysis indicators and tools
import joblib  # For model serialization and deserialization

# Google Cloud imports
from google.cloud import storage  # For interacting with Google Cloud Storage
from google.oauth2 import service_account  # For Google Cloud authentication

# Empyreal SDK imports for strategy development and backtesting
from emp_orderly import Strategy, EmpOrderly  # For strategy implementation and management
from emp_orderly_types import PerpetualAssetType, Interval  # For defining asset types and intervals

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

/workspaces/orderly-x-ai-trading-agent/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Class implementing the trading strategy
class AlphaWaveTrader(Strategy):
    def init(self):
        """
        Initialize the strategy by loading the pre-trained model and scaler from Google Cloud Storage.
        Pre-compute technical indicators used to generate buy/sell signals.
        """
        # Load GCP credentials and project ID from environment variables
        credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
        project_id = os.getenv("GCP_PROJECT_ID")

        # Ensure required environment variables are set
        if not project_id or not credentials_path:
            raise ValueError("GCP_PROJECT_ID and GOOGLE_APPLICATION_CREDENTIALS must be set in the .env file.")

        # Initialize Google Cloud service account credentials
        credentials = service_account.Credentials.from_service_account_file(credentials_path)

        # Initialize Google Cloud Storage client
        storage_client = storage.Client(credentials=credentials, project=project_id)

        # Access the GCS bucket containing the model and scaler files
        bucket = storage_client.bucket(f'{project_id}-crypto_trading_bucket')

        # Download and load the pre-trained model
        model_blob = bucket.blob('trading_model.pkl')
        model_blob.download_to_filename('trading_model.pkl')
        self.model = joblib.load('trading_model.pkl')

        # Download and load the scaler
        scaler_blob = bucket.blob('scaler.pkl')
        scaler_blob.download_to_filename('scaler.pkl')
        self.scaler = joblib.load('scaler.pkl')

        # Prepare DataFrame with technical indicators for strategy signals
        df = pd.DataFrame({
            'close': self.data.close,
            'high': self.data.high,
            'low': self.data.low
        })

        # Add technical indicators
        df['SMA_10'] = ta.sma(df['close'], length=10)
        df['SMA_20'] = ta.sma(df['close'], length=20)
        df['RSI'] = ta.rsi(df['close'], length=14)

        # MACD and Bollinger Bands
        macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_signal'] = macd['MACDs_12_26_9']
        bbands = ta.bbands(df['close'], length=20)
        df['BB_upper'] = bbands['BBU_20_2.0']
        df['BB_middle'] = bbands['BBM_20_2.0']
        df['BB_lower'] = bbands['BBL_20_2.0']

        # Additional indicators
        df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)
        df['MOM'] = ta.mom(df['close'], length=10)
        df['ROC'] = ta.roc(df['close'], length=10)

        # Drop NaN values
        df.dropna(inplace=True)

        # Store indicator data for later use in the strategy
        self.indicator_data = df.reset_index(drop=True)

    def next(self):
        """
        Generate buy/sell signals based on the model's prediction using the latest data.
        """
        # Get the current index of the data point
        idx = len(self.data.close) - 1

        # Ensure there is enough data to make predictions
        if idx < len(self.indicator_data):
            # Extract the technical indicators for the current index
            row = self.indicator_data.iloc[idx]

            # Define the feature set for the model
            features = [
                row['SMA_10'], row['SMA_20'], row['RSI'], row['MACD'], row['MACD_signal'],
                row['BB_upper'], row['BB_middle'], row['BB_lower'], row['ATR'], row['MOM'], row['ROC']
            ]

            # Scale the features using the pre-loaded scaler
            scaled_features = self.scaler.transform([features])

            # Get the model's prediction
            prediction = self.model.predict(scaled_features)[0]

            # Implement buy/sell logic based on the model's prediction
            if prediction == 1:
                # Buy signal: close short positions and open long
                if self.position.is_short:
                    self.position.close()
                if not self.position.is_long:
                    self.buy(size=1)
            else:
                # Sell signal: close long positions and open short
                if self.position.is_long:
                    self.position.close()
                if not self.position.is_short:
                    self.sell(size=1)

In [12]:
# Main function for backtesting the strategy
async def main():
    """
    Main function to configure the SDK, load the custom strategy, and backtest it using historical data.
    """
    # Initialize the SDK with backtesting parameters
    sdk = EmpOrderly(
        cash=4000,  # Initial cash for backtesting
        commission=0.0001,  # Commission rate per trade
        exclusive_orders=True  # Ensure only one position is active at a time
    )

    # Set the strategy and load historical data
    sdk.set_strategy(AlphaWaveTrader)

    # Load historical data with specified lookback and asset type
    await sdk.load_data(
        lookback=60,  # value to be 14
        interval=Interval.four_hour, # 30m here
        asset=PerpetualAssetType.ETH,
    )

    # Perform backtesting
    stats = sdk.backtest()
    print(stats)


# Entry point of the script
if __name__ == "__main__":
    # Apply nest_asyncio to avoid runtime issues with asynchronous calls
    nest_asyncio.apply()

    # Run the main asynchronous function until it completes
    asyncio.get_event_loop().run_until_complete(main())

Start                     2024-08-14 08:00:00
End                       2024-10-13 04:00:00
Duration                     59 days 20:00:00
Exposure Time [%]                   99.444444
Equity Final [$]                  5229.520004
Equity Peak [$]                    5360.32777
Return [%]                             30.738
Buy & Hold Return [%]              -10.406334
Return (Ann.) [%]                  387.548129
Volatility (Ann.) [%]              143.318591
Sharpe Ratio                         2.704102
Sortino Ratio                       31.547969
Calmar Ratio                        53.319023
Max. Drawdown [%]                   -7.268478
Avg. Drawdown [%]                   -1.496357
Max. Drawdown Duration       17 days 16:00:00
Avg. Drawdown Duration        2 days 04:00:00
# Trades                                   53
Win Rate [%]                        54.716981
Best Trade [%]                      12.169839
Worst Trade [%]                     -4.150778
Avg. Trade [%]                    